<h2>Initialise files</h2>

In [ ]:
import pandas as pd
import math
from functools import reduce

Declare variables constant

In [ ]:
FILE_PATH = "2021-06-27-AccountStatement.csv"
TABLE_NAMES = ['Cash Balance', 'Account Trade History', 'Equities', 'Options', 'Profits and Losses']

def parse_key(key): 
    return key.lower().replace(" ", "_")

def build_map(acc, cur): 
    new_acc = {parse_key(acc): acc} if isinstance(acc, str) else acc  
    key = parse_key(cur)
    new_acc[key] = cur
    return new_acc

table_map = reduce(build_map, TABLE_NAMES)   
print(table_map)

table_length = len(TABLE_NAMES)

def parse_row_list(row_list, table_name): 
    """
    Remove last row for the tables with total, 
    as this will disrupt with the data transformation process.
    i.e.  cash balance, equities and profits and losses 
    """
    table_names_to_remove = ['Cash Balance', 'Equities', 'Profits and Losses']
    
    if table_name in table_names_to_remove: 
        row_list.pop()
        return row_list 
    
    return row_list
    

In [ ]:
import csv

def split_csv(csv_path, table_names): 
    tables_infos = detect_tables_from_csv(csv_path, table_names)
    
def create_output_file_and_return_writer(file_name): 
    row_list = []
    output_file = open(f'generated/{file_name}.csv', 'w',  newline='')
    writer =  csv.writer(output_file)
    return [row_list, output_file, writer]
    
def detect_tables_from_csv(csv_path, table_names): 
    file = open(csv_path, 'r')
    csv_reader = csv.reader(file)
    
    current_table_idx = 0
    [row_list, output_file, writer] = create_output_file_and_return_writer(TABLE_NAMES[current_table_idx])
    push_to_table = False 
    
    for  row in csv_reader: 
        if len(row) == 0: 
            if push_to_table == True:
                push_to_table = False
                parsed_rl = parse_row_list(row_list, TABLE_NAMES[current_table_idx])
                writer.writerows(parsed_rl)
                output_file.close()
                current_table_idx += 1 
                if(current_table_idx < table_length):
                    [row_list, output_file, writer] = create_output_file_and_return_writer(TABLE_NAMES[current_table_idx])
            continue
        
        first_data = row[0]
        if push_to_table == True: 
            # for some reasons all the 
            row_itme = first_data.split(',')
            row_list.append(row)
            continue
        
            
        if first_data in TABLE_NAMES: 
            push_to_table = True
            continue 

In [ ]:
split_csv(FILE_PATH, TABLE_NAMES)

<h2>Create individual ticker chart</h2>

<strong>Initialise dataframe</strong>

In [ ]:
FILE_PATH = f'generated/{table_map["cash_balance"]}.csv'
df = pd.read_csv(FILE_PATH)
df.info()

In [ ]:
def convert_currency(value): 
    """
    Convert string number value to float 
    - Remove commas 
    - Convert to float type
    """
    if isinstance(value, float): 
        return value
    
    new_val = value.replace(',','')
    return float(new_val)

<h3>Parse description into relevant columns</h3>

In [ ]:
'''
example description: 
SOLD -1 VERTICAL PINS 100 19 FEB 21 60/55 PUT @.94 PHLX 
BOT +5 BABA @229.40
SOLD -1 NIO 100 19 FEB 21 40 PUT @2.23 BOX
'''
# net_qty refers to the first qty with the +|- symbol
net_qty_regex = '((\+|-)\d+)'

# qty_with_weekly refers to the numerical qty just before the date
qty_with_weekly_regex = f'\s\d+(?:\s\(Weeklys\))?'
date_regex = '\s(([0][1-9]|[12]?\d|3[01])\s(JAN|FEB|MA[RY]|APR|JU[NL]|AUG|SEPT?|OCT|NOV|DEC)\s(\d{2}))'
symbol_regex = f'\s([A-Z]+){qty_with_weekly_regex}{date_regex}'
spread_regex = f'\d\s([A-Z\s]+){symbol_regex}'

df['Datetime'] = pd.to_datetime(df['DATE'] +' ' + df['TIME'])
df['AMOUNT'] = df['AMOUNT'].apply(lambda x: x if isinstance(x, str) == False and math.isnan(x) else convert_currency(x)) 

df['Symbol'] = df['DESCRIPTION'].str.extract(symbol_regex, expand=True)[0]
df['Spread'] = df['DESCRIPTION'].str.extract(spread_regex, expand=True)[0]
df['Qty'] = df['DESCRIPTION'].str.extract(net_qty_regex, expand=True)[0]
df['Exp'] =  df['DESCRIPTION'].str.extract(date_regex, expand=True)[0]

df.head()

In [ ]:
df.info()

In [ ]:
unique_tickers = df['Symbol'].dropna().unique()
print(unique_tickers)

<strong>Initialise variables</strong>

In [ ]:
TICKER = "TSLA"

<strong>Retrieve chart data</strong>

In [ ]:
ticker_data = df[df['Symbol'] == TICKER]
ticker_data.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import waterfall_chart

def bar_chart(df): 
    fig = plt.figure(figsize=(15, 4))
    sns.barplot(x="Datetime", y="AMOUNT", data=df)
    plt.xlabel('Datetime')
    plt.xticks(rotation=90)
    plt.ylabel('P&L')
    plt.show()
    
bar_chart(ticker_data)

In [ ]:
'''
Copied from https://github.com/chrispaulca/waterfall/blob/master/waterfall_chart.py
If we are to use the installed package, for some reason fig size can't be change, so we run the code here. 
'''

'''
A function that attempts to generate a standard waterfall chart in generic Python. Requires two sequences,
one of labels and one of values, ordered accordingly.
'''


from matplotlib.ticker import FuncFormatter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as lines

#------------------------------------------

def plot(index, data, Title="", x_lab="", y_lab="",
              formatting = "{:,.1f}", green_color='#29EA38', red_color='#FB3C62', blue_color='#24CAFF',
             sorted_value = False, threshold=None, other_label='other', net_label='net', 
             rotation_value = 30, blank_color=(0,0,0,0), figsize=(10,10)):
    '''
    Given two sequences ordered appropriately, generate a standard waterfall chart.
    Optionally modify the title, axis labels, number formatting, bar colors, 
    increment sorting, and thresholding. Thresholding groups lower magnitude changes
    into a combined group to display as a single entity on the chart.
    '''
    
    #convert data and index to np.array
    index=np.array(index)
    data=np.array(data)
    
    # wip
    #sorted by absolute value 
    if sorted_value: 
        abs_data = abs(data)
        data_order = np.argsort(abs_data)[::-1]
        data = data[data_order]
        index = index[data_order]
    
    #group contributors less than the threshold into 'other' 
    if threshold:
        
        abs_data = abs(data)
        threshold_v = abs_data.max()*threshold
        
        if threshold_v > abs_data.min():
            index = np.append(index[abs_data>=threshold_v],other_label)
            data = np.append(data[abs_data>=threshold_v],sum(data[abs_data<threshold_v]))
    
    changes = {'amount' : data}
    
    #define format formatter
    def money(x, pos):
        'The two args are the value and tick position'
        return formatting.format(x)
    formatter = FuncFormatter(money)
    
    fig, ax = plt.subplots(figsize=figsize)
    ax.yaxis.set_major_formatter(formatter)

    #Store data and create a blank series to use for the waterfall
    trans = pd.DataFrame(data=changes,index=index)
    blank = trans.amount.cumsum().shift(1).fillna(0)
    
    trans['positive'] = trans['amount'] > 0

    #Get the net total number for the final element in the waterfall
    total = trans.sum().amount
    trans.loc[net_label]= total
    blank.loc[net_label] = total

    #The steps graphically show the levels as well as used for label placement
    step = blank.reset_index(drop=True).repeat(3).shift(-1)
    step[1::3] = np.nan

    #When plotting the last element, we want to show the full bar,
    #Set the blank to 0
    blank.loc[net_label] = 0
    
    #define bar colors for net bar
    trans.loc[trans['positive'] > 1, 'positive'] = 99
    trans.loc[trans['positive'] < 0, 'positive'] = 99
    trans.loc[(trans['positive'] > 0) & (trans['positive'] < 1), 'positive'] = 99
    
    trans['color'] = trans['positive']
    
    trans.loc[trans['positive'] == 1, 'color'] = green_color
    trans.loc[trans['positive'] == 0, 'color'] = red_color
    trans.loc[trans['positive'] == 99, 'color'] = blue_color
    
    my_colors = list(trans.color)
    
    #Plot and label
    my_plot = plt.bar(range(0,len(trans.index)), blank, width=0.5, color=blank_color)
    plt.bar(range(0,len(trans.index)), trans.amount, width=0.6,
             bottom=blank, color=my_colors)       
                                   
    
    # connecting lines - figure out later
    #my_plot = lines.Line2D(step.index, step.values, color = "gray")
    #my_plot = lines.Line2D((3,3), (4,4))
    
    #axis labels
    plt.xlabel("\n" + x_lab)
    plt.ylabel(y_lab + "\n")

    #Get the y-axis position for the labels
    y_height = trans.amount.cumsum().shift(1).fillna(0)
    
    temp = list(trans.amount)
    
    # create dynamic chart range
    for i in range(len(temp)):
        if (i > 0) & (i < (len(temp) - 1)):
            temp[i] = temp[i] + temp[i-1]
    
    trans['temp'] = temp
            
    plot_max = trans['temp'].max()
    plot_min = trans['temp'].min()
    
    #Make sure the plot doesn't accidentally focus only on the changes in the data
    if all(i >= 0 for i in temp):
        plot_min = 0
    if all(i < 0 for i in temp):
        plot_max = 0
    
    if abs(plot_max) >= abs(plot_min):
        maxmax = abs(plot_max)   
    else:
        maxmax = abs(plot_min)
        
    pos_offset = maxmax / 40
    
    plot_offset = maxmax / 15 ## needs to me cumulative sum dynamic

    #Start label loop
    loop = 0
    for index, row in trans.iterrows():
        # For the last item in the list, we don't want to double count
        if row['amount'] == total:
            y = y_height[loop]
        else:
            y = y_height[loop] + row['amount']
        # Determine if we want a neg or pos offset
        if row['amount'] > 0:
            y += (pos_offset*2)
            plt.annotate(formatting.format(row['amount']),(loop,y),ha="center", color = 'g', fontsize=9)
        else:
            y -= (pos_offset*4)
            plt.annotate(formatting.format(row['amount']),(loop,y),ha="center", color = 'r', fontsize=9)
        loop+=1

    #Scale up the y axis so there is room for the labels
    plt.ylim(plot_min-round(3.6*plot_offset, 7),plot_max+round(3.6*plot_offset, 7))
    
    #Rotate the labels
    plt.xticks(range(0,len(trans)), trans.index, rotation=rotation_value)
    
    #add zero line and title
    plt.axhline(0, color='black', linewidth = 0.6, linestyle="dashed")
    plt.title(Title)
    plt.tight_layout()

    return fig, ax

In [ ]:
index = ticker_data['Datetime'].values
data = ticker_data['AMOUNT'].values

waterfall_plot = plot(index=index, data=data, rotation_value=90, Title=TICKER, x_lab="Datetime", y_lab="Amount", figsize=(18,8))